In [1]:
# imports
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import math
import keras
from scipy.stats  import norm
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras        import backend as K
from datetime     import datetime
from keras.optimizers      import RMSprop
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics       import mean_squared_error
from matplotlib.pylab      import rcParams
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
%matplotlib inline
rcParams['figure.figsize'] = 15, 6

# number of inputs to be fed
look_back    = 288
# number of outputs to be generated
look_forward = 24
# the number of stations
stations     = 66

Using TensorFlow backend.


In [2]:
# The interval between each dataset (original data in 5 minute intervals)
time_grouping = '5min'
# load the dataset
dataframe = pd.read_csv('ignored_assets/paxout_table.csv', engine='python', nrows=288*21)
dataframe['time_bucket'] = pd.to_datetime(dataframe['time_bucket'])
dataframe = dataframe.set_index('time_bucket')
# dataframe['total'] = dataframe.sum(axis=1)
dataframe['day_hour'] = dataframe.index.round(time_grouping)
dataframe = dataframe.groupby('day_hour').sum()
# removes the timestamp at column 67
dataset_orig = dataframe.values[:,:stations]
dataset_orig = dataset_orig.astype('float32')
# scale the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset_orig)

# convert an array of values into a dataset matrix, adjusted to make a dateset that is 66 wide
def create_dataset(dataset, look_back=1, look_forward=2):
    dataX, dataY = [], []
    np.array(dataY)
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back),:]
        # Makes sure that the array isn't the last 2 or 3 or whatever bits. It has to be the full 24
        if len(dataset[i + look_back:i+look_back+look_forward, 0]) == look_forward:
            dataX.append(a.T)
            dataY.append(dataset[i + look_back:i+look_back+look_forward, :].T)
    return np.array(dataX), np.array(dataY)

# split into train and test sets
train_size = 288*10
test_size = len(dataset) - train_size
test, train = dataset[0:test_size,:], dataset[test_size:len(dataset)-look_forward,:]

# reshape into X=[t, t-1, t-2,..., t-look_back] and Y=[t+1, t+2,... t+look_forward]
trainX, trainY = create_dataset(train, look_back, look_forward)

In [3]:
def top_heavy_loss (y_true, y_pred):
    w = np.arange(1,0, -1./24) ** 2
    w = (w / w.sum())[:,None]
    W = K.variable(value = w)
    return K.dot( K.abs(y_pred-y_true), W)

In [ ]:

# creates a model but does not compile the model
def create_model(dropout_rate      = 0.1):
    weight_constraint = 1
    layers = 6
    neurons = 64
    model = Sequential()
    model.add(Dropout(dropout_rate, input_shape=(stations, look_back)))
    lstm = LSTM(128, recurrent_dropout=dropout_rate, return_sequences=True)
    model.add(lstm)
    # 6 Layers, 64 Neurons
    for i in range(layers):
        model.add(Dropout(dropout_rate))
        model.add(Dense(neurons))
    model.add(Dense(look_forward))
    model.compile(loss=top_heavy_loss, optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0), metrics=['accuracy'])
    return model

# creates the model
model = KerasRegressor(build_fn=create_model, epochs=60, batch_size=40, verbose=3)

# The Parameters to Search Through
dropout_rate  = [0.1, 0.2, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8]

# Defines it as a dictionary
param_grid = dict(dropout_rate = dropout_rate)

# Begins the Search For the Optimal Properties
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=12)

grid_result = grid.fit(trainX, trainY)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 1/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 2/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 3/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 4/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 6/60
Epoch 6/60
Epoch 6/60
Epoch 6/60
Epoch 6/60
Epoch 5/60
Epoch 6/60
Epoch 6/60
Epoch 6/60
Epoch 6/60
Epoch 6/60
Epoch 6/60
Epoch 7/60
Epoch 7/60
Epoch 7/60
Epoch 7/60
Epoch 7/60
Epoch 6/60
Epoch 7/60
Epoch 7/60
Epoch 7/60
Epoch 7/60
Epoch 7/60
Epoch 7/60
Epoch 8/60
Epoch 8/60
Epoch 8/60
Epoch 8/60
Epoch 8/60
Epoch 7/60
Epoch 8/60
Epoch 8/60

Epoch 58/60
Epoch 59/60
Epoch 58/60
Epoch 59/60
Epoch 58/60
Epoch 59/60
Epoch 59/60
Epoch 59/60
Epoch 59/60
Epoch 58/60
Epoch 59/60
Epoch 59/60
Epoch 59/60
Epoch 59/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 59/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 59/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 2/60
Epoch 2/60
Epoch 1/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 3/60
Epoch 3/60
Epoch 1/60
Epoch 3/60
Epoch 2/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 2/60
Epoch 4/60
Epoch 3/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 4/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 3/60
Epoch 5/60
Epoch 4/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 5/60
Epoch 6/60
Epoch 6/60
Epoch 6/60
Epoc

Epoch 56/60
Epoch 57/60
Epoch 56/60
Epoch 56/60
Epoch 56/60
Epoch 56/60
Epoch 55/60
Epoch 56/60
Epoch 57/60
Epoch 57/60
Epoch 57/60
Epoch 58/60
Epoch 57/60
Epoch 57/60
Epoch 57/60
Epoch 57/60
Epoch 57/60
Epoch 57/60
Epoch 56/60
Epoch 58/60
Epoch 57/60
Epoch 58/60
Epoch 58/60
Epoch 58/60
Epoch 59/60
Epoch 58/60
Epoch 58/60
Epoch 58/60
Epoch 58/60
Epoch 58/60
Epoch 57/60
Epoch 59/60
Epoch 59/60
Epoch 58/60
Epoch 59/60
Epoch 60/60
Epoch 59/60
Epoch 59/60
Epoch 59/60
Epoch 59/60
Epoch 59/60
Epoch 59/60
Epoch 58/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 59/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 60/60
Epoch 59/60
Epoch 60/60
Epoch 60/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 1/60
Epoch 2/60
Epoch 1/60
Epoch 1/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 3/60
Epoch 2/60
Epoch 2/60
Epoch 2/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 4/60
Epoch 3/60
Epoch 3/60
Epoch 3/60
Epoch 4/60
Epoch 4/6

In [ ]:
'''
Given: 
* epochs = 10
* batch_size = 60


Best: 0.073481 using {'dropout_rate': 0.5}
0.068988 (0.001514) with: {'dropout_rate': 0.1}
0.068582 (0.002472) with: {'dropout_rate': 0.2}
0.069281 (0.002085) with: {'dropout_rate': 0.3}
0.070244 (0.002457) with: {'dropout_rate': 0.4}
0.073481 (0.004081) with: {'dropout_rate': 0.5}

Best: 0.073881 using {'dropout_rate': 0.5}
0.068489 (0.002157) with: {'dropout_rate': 0.1}
0.068176 (0.002022) with: {'dropout_rate': 0.2}
0.069107 (0.002549) with: {'dropout_rate': 0.3}
0.070668 (0.003781) with: {'dropout_rate': 0.4}
0.073881 (0.003803) with: {'dropout_rate': 0.5}

Best: 0.073639 using {'dropout_rate': 0.5}
0.069010 (0.002797) with: {'dropout_rate': 0.2}
0.069681 (0.002601) with: {'dropout_rate': 0.3}
0.069333 (0.002836) with: {'dropout_rate': 0.35}
0.070192 (0.003108) with: {'dropout_rate': 0.4}
0.071822 (0.003420) with: {'dropout_rate': 0.45}
0.073639 (0.003678) with: {'dropout_rate': 0.5}

Best: 0.095888 using {'dropout_rate': 0.7}
0.068534 (0.002568) with: {'dropout_rate': 0.3}
0.070097 (0.003232) with: {'dropout_rate': 0.4}
0.073350 (0.003495) with: {'dropout_rate': 0.5}
0.080014 (0.005029) with: {'dropout_rate': 0.6}
0.095888 (0.007266) with: {'dropout_rate': 0.7}
'''

In [5]:
np.random.seed(4)

# creates a model but does not compile the model
def create_model(layers = 3,
                 neurons = 128):
    dropout_rate = 0.3
    model = Sequential()
    model.add(Dropout(dropout_rate, input_shape=(stations, look_back)))
    lstm = LSTM(neurons, recurrent_dropout=dropout_rate, return_sequences=True)
    model.add(lstm)
    for i in range(layers):
        model.add(Dropout(dropout_rate))
        model.add(Dense(neurons))
    model.add(Dense(look_forward))
    model.compile(loss=top_heavy_loss, optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0), metrics=['accuracy'])
    return model

# creates the model
model = KerasRegressor(build_fn=create_model, epochs=10, batch_size=60, verbose=5)

# The Parameters to Search Through
layers = [5, 6]
neurons  = [64, 128, 192]

# Defines it as a dictionary
param_grid = dict(layers = layers, neurons = neurons)

# Begins the Search For the Optimal Properties
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=12)

grid_result = grid.fit(trainX, trainY)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 2/10
Epoch 2/10
Epoch 2/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 3/10
Epoch 3/10
Epoch 3/10
Epoch 2/10
Epoch 2/10
Epoch 2/10
Epoch 2/10
Epoch 2/10
Epoch 2/10
Epoch 4/10
Epoch 4/10
Epoch 4/10
Epoch 3/10
Epoch 3/10
Epoch 3/10
Epoch 5/10
Epoch 5/10
Epoch 5/10
Epoch 2/10
Epoch 3/10
Epoch 2/10
Epoch 3/10
Epoch 2/10
Epoch 4/10
Epoch 3/10
Epoch 4/10
Epoch 6/10
Epoch 4/10
Epoch 6/10
Epoch 6/10
Epoch 5/10
Epoch 7/10
Epoch 5/10
Epoch 7/10
Epoch 5/10
Epoch 7/10
Epoch 4/10
Epoch 4/10
Epoch 4/10
Epoch 8/10
Epoch 6/10
Epoch 8/10
Epoch 6/10
Epoch 6/10
Epoch 8/10
Epoch 3/10
Epoch 9/10
Epoch 3/10
Epoch 3/10
Epoch 9/10
Epoch 7/10
Epoch 7/10
Epoch 7/10
Epoch 9/10
Epoch 5/10
Epoch 5/10
Epoch 10/10
Epoch 5/10
Epoch 10/10
Epoch 8/10
Epoch 8/10
Epoch 8/10
Epoch 10/10
Epoch 9/10
Epoch 9/10
Epoch 9/10
Epoch 6/10
Epoch 6/10
Epoch 6/10
Epoch 4/10
Epoch 10/10
Epoch 4/10
Epoch 4/10
Epoch 10/10
Epoch 10/10
Epoc

In [ ]:
'''

With Parameters
* Epochs 10
* Batch 60
* Dropout 0.1

Best: 0.069670 using {'layers': 4, 'neurons': 128}
0.068881 (0.002203) with: {'layers': 2, 'neurons': 128}
0.067406 (0.002277) with: {'layers': 2, 'neurons': 256}
0.068179 (0.002432) with: {'layers': 3, 'neurons': 128}
0.068775 (0.002802) with: {'layers': 3, 'neurons': 256}
0.069670 (0.004567) with: {'layers': 4, 'neurons': 128}
0.068349 (0.003953) with: {'layers': 4, 'neurons': 256}

Best: 0.071536 using {'layers': 5, 'neurons': 128}
0.068984 (0.001735) with: {'layers': 3, 'neurons': 128}
0.068473 (0.003188) with: {'layers': 3, 'neurons': 256}
0.071083 (0.001236) with: {'layers': 4, 'neurons': 128}
0.067224 (0.001717) with: {'layers': 4, 'neurons': 256}
0.071536 (0.001275) with: {'layers': 5, 'neurons': 128}
0.067902 (0.001421) with: {'layers': 5, 'neurons': 256}

Best: 0.071820 using {'layers': 6, 'neurons': 64}
0.070832 (0.002609) with: {'layers': 4, 'neurons': 64}
0.068442 (0.002627) with: {'layers': 4, 'neurons': 128}
0.070888 (0.002275) with: {'layers': 5, 'neurons': 64}
0.068542 (0.002012) with: {'layers': 5, 'neurons': 128}
0.071820 (0.002721) with: {'layers': 6, 'neurons': 64}
0.068779 (0.002276) with: {'layers': 6, 'neurons': 128}

With Parameters
* Epochs 10
* Batch 60
* Dropout 0.3

Best: 0.072169 using {'layers': 6, 'neurons': 128}
0.069883 (0.002344) with: {'layers': 3, 'neurons': 128}
0.069544 (0.002408) with: {'layers': 3, 'neurons': 256}
0.071780 (0.003125) with: {'layers': 4, 'neurons': 128}
0.067031 (0.001918) with: {'layers': 4, 'neurons': 256}
0.072045 (0.002474) with: {'layers': 5, 'neurons': 128}
0.068107 (0.002212) with: {'layers': 5, 'neurons': 256}
0.072169 (0.002966) with: {'layers': 6, 'neurons': 128}
0.068203 (0.002480) with: {'layers': 6, 'neurons': 256}

Best: 0.080197 using {'layers': 6, 'neurons': 64}
0.078916 (0.004541) with: {'layers': 5, 'neurons': 64}
0.071914 (0.003064) with: {'layers': 5, 'neurons': 128}
0.070201 (0.004656) with: {'layers': 5, 'neurons': 192}
0.080197 (0.004459) with: {'layers': 6, 'neurons': 64}
0.071463 (0.002642) with: {'layers': 6, 'neurons': 128}
0.070445 (0.002954) with: {'layers': 6, 'neurons': 192}

Best: 0.080195 using {'layers': 6, 'neurons': 64}
0.078933 (0.004554) with: {'layers': 5, 'neurons': 64}
0.071911 (0.003069) with: {'layers': 5, 'neurons': 128}
0.070286 (0.004772) with: {'layers': 5, 'neurons': 192}
0.080195 (0.004467) with: {'layers': 6, 'neurons': 64}
0.071259 (0.002701) with: {'layers': 6, 'neurons': 128}
0.071727 (0.005488) with: {'layers': 6, 'neurons': 192}
'''